# Populate model from excel

In [1]:
import capellambse

import ipywidgets as widgets
from IPython.core.display import HTML
import numpy as np
import pandas as pd 
from IPython.display import Markdown


resources = {
    "Import-Test": "Import-Test/Import Test",
}
path_to_model = "../Import Test.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)

import capellambse_helper #local python module in same directory as notebook needs to be included after model object is created since it patches/extends capabilitits of the model 

In [2]:
import io
from capellambse import decl
pa_model = model.pa 

capellambse_helper.display_function_property_values(pa_model)

capellambse_helper.display_component_property_values(pa_model)

#help(la_model.root_component)

# Example system components you want to create

import pandas as pd

class ExcelDataFrameReader:
    def __init__(self, file_path):
        """
        Initialize the ExcelDataFrameReader with the path to the Excel file.
        """
        self.file_path = file_path
        self.df = None

    def load_data(self):
        """
        Load the Excel file into a Pandas DataFrame.
        """
        try:
            self.df = pd.read_excel(self.file_path)
            print(f"Data successfully loaded from {self.file_path}")
        except Exception as e:
            print(f"Error loading data: {e}")

    def list_rows(self):
        """
        Iterate through the DataFrame rows and print each row's fields.
        """
        if self.df is not None:
            for index, row in self.df.iterrows():
                print(f"Row {index}:")
                print(f"Tc_Level: {row['Tc_Level']}")
                print(f"Tc_ObjectType: {row['Tc_ObjectType']}")
                print(f"Title: {row['Title']}")
                print("-" * 40)  # Separator for readability
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")

    def get_row(self, index):
        """
        Get the row by index as a dictionary.
        """
        if self.df is not None:
            if index < len(self.df):
                return self.df.iloc[index].to_dict()
            else:
                print(f"Index {index} is out of bounds.")
                return None
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")
            return None

    def get_column(self, column_name):
        """
        Get all data from a specific column.
        """
        if self.df is not None:
            if column_name in self.df.columns:
                return self.df[column_name].tolist()
            else:
                print(f"Column {column_name} does not exist.")
                return None
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")
            return None

    def get_value(self, index, column_name):
        """
        Get a specific value from the DataFrame by row index and column name.
        """
        if self.df is not None:
            if index < len(self.df) and column_name in self.df.columns:
                return self.df.at[index, column_name]
            else:
                print(f"Either index {index} is out of bounds or column {column_name} does not exist.")
                return None
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")
            return None

    def get_next_row(self):
        """
        Get the next row starting from the current index and update the current index.
        """
        if self.df is not None:
            if self.current_index < len(self.df):
                self.current_index += 1
                row_data = self.df.iloc[self.current_index].to_dict()
                return None
            else:
                print("No more rows available.")
                return None
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")
            return None

    def get_current_row(self):
        """
        Get the row by index as a dictionary.
        """
        if self.df is not None:
            if self.current_index < len(self.df) -1:
                print( self.df.iloc[self.current_index].to_dict()["Title"], "rcl:" ,self.df.iloc[self.current_index].to_dict()["Tc_Level"])
                return self.df.iloc[self.current_index].to_dict()
            else:
                print(f"Index {self.current_index} is out of bounds.")
                print(f"Index providing last row")
                #return self.df.iloc[self.current_index].to_dict()
                return self.df.iloc[len(self.df) -2].to_dict()
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")
            return None
    
    def get_first_row(self):
        """
        Reset the pointer to the first row.
        """
        self.current_index = 0
        print("Current row pointer has been reset to the first row.")
        row_data = self.df.iloc[self.current_index].to_dict()
        return row_data
    
    def is_last_row(self):
        """
        Check if the current index points to the last row.
        """
        if self.df is not None:
            return (self.current_index)  == (len(self.df) -1)
        else:
            print("Data has not been loaded. Please call the 'load_data()' method first.")
            return False

# Example usage:
file_path = 'ground vehicle 2.xlsx'  # Replace with your actual path

# Instantiate the class
excel_content = ExcelDataFrameReader(file_path)

# Load the data
excel_content.load_data()

# Access specific row data
row_2 = excel_content.get_row(2)
print(f"Row 2: {row_2}")

# Access a specific column's data
tc_level_column = excel_content.get_column('Tc_Level')
print(f"Tc_Level column: {tc_level_column}")

# Access a specific value from a row and column
value = excel_content.get_value(2, 'Title')
print(f"Value in row 2, 'Title' column: {value}")

#excel_content.list_rows()
#import capellambse.metamodel.cs
#import capellambse.loader.core
from capellambse.metamodel.cs import Part
#from capellambse.loader.core import MelodyLoader,FragmentType
#import capellambse.loader.core 

def Create_Component_Level(current_level,parent,excel_content):
    print()
    print("Entering","cl",current_level)

    while  excel_content.is_last_row() == False and   int(excel_content.get_current_row()["Tc_Level"]) >= current_level :
        print("row:",excel_content.get_current_row())
        print("last_row",excel_content.is_last_row())
        print("current row :", excel_content.get_current_row()["Title"], "rcl:" ,int(excel_content.get_current_row()["Tc_Level"]),"cl:",current_level)
        
        if   int(excel_content.get_current_row()["Tc_Level"]) > current_level :  #assume only one jump in level.        
            current_level = int(excel_content.get_current_row()["Tc_Level"])
            parent = new_lc                  
            Create_Component_Level(current_level,parent,excel_content)
            current_level = int(excel_content.get_current_row()["Tc_Level"])
        else :
            current_level = int(excel_content.get_current_row()["Tc_Level"])
            print(parent)
            new_lc = parent.owned_components.create(name = excel_content.get_current_row()['Title'] )
            new_lc.nature   = 'BEHAVIOUR'
            nnew_lc.kind     = 'UNSET'
            parent.owned_features.create("Part", name= new_lc.name, type=new_lc)          
            print("created :", excel_content.get_current_row()["Title"])
            excel_content.get_next_row() 
        print()
    print("Exiting",current_level)

#print(la_model.component_package)
#help(la_model.component_package)
#print(la_model.component_package)
#print("1")
#print(la_model.component_package.components[0].components[0])
#print("2")
#print(la_model.component_package.components[0].components[1])
#print("3")

       
#print(la_model.component_package.components[1])
#print(la_model.component_package.components[2])
#print()
#parent = la_model.root_component
#print(parent)
#print()

parent = pa_model.root_component
print(parent)
help(parent)
print()
print(parent.parts[0])
help(parent.parts[0])
print(dir(capellambse.metamodel.cs))

print()
row = excel_content.get_first_row()
current_level = 0

Create_Component_Level(current_level,parent,excel_content)

#model.save()

,Logical Function,Property Value Group Name,Propery Name,Property Value


,Logical Component,Property Value Group Name,Property Name,Property Value


Data successfully loaded from ground vehicle 2.xlsx
Row 2: {'Tc_Level': 2, 'Tc_ObjectType': 'Fnd0LogicalBlock', 'Title': 'Lead Variant   '}
Tc_Level column: [0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 1, 2, 3, 3, 2, 3, 3, 2, 2, 1, 2, 2, 2, 1, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, '<endtag>']
Value in row 2, 'Title' column: Lead Variant   
<PhysicalComponent 'Physical System' (26b52154-a582-4936-979e-3a761ef0aa79)>
.allocated_functions = []
.applied_property_value_groups = []
.applied_property_values = []
.components = []
.constraints = []
.context_diagram = <Diagram 'Context of Physical System'>
.deployed_components = []
.deploying_components = ... # backreference to PhysicalComponent - omitted: can be slow to compute
.description = Markup('')
.diagrams = []
.exchanges = []
.filtering_cri

KeyError: 'BEHAVIOUR'